# 채권 분석 보고서 크롤링

In [15]:
# 채권 분석자료 Crawling

import requests
from bs4 import BeautifulSoup
from tika import parser
import pandas as pd
from IPython.display import clear_output
    
start_page = 1
current_page = start_page
end_page = 126

result_df = pd.DataFrame(columns=['date', 'text'])

while True:
    URL = 'https://finance.naver.com/research/debenture_list.nhn?&page={}'.format(current_page)
    r = requests.get(URL)
    sp = BeautifulSoup(r.text, 'html.parser')
    results = sp.select('table.type_1 td.file')

    for i, result in enumerate(results):
        pdf_date = sp.select('table.type_1 td.date')[i*2].text
        pdf_temp = requests.get(result.select_one('a').attrs['href'])

        with open('bond.pdf', 'wb') as f:
            f.write(pdf_temp.content)
        
        parsed = parser.from_file('bond.pdf')
        temp_text = str(parsed['content']).strip()
        temp_df = pd.DataFrame([{'date':pdf_date, 'text':temp_text}])
        result_df = pd.concat([result_df, temp_df]) 
        
    clear_output(wait=True)
    print('크롤링 {}% 진행 중...'.format(int((current_page/end_page)*100)))
        
    if current_page == end_page:
        clear_output(wait=True)
        print('데이터 크롤링 완료 ! bond_datas.csv에서 확인 가능')
        result_df.to_csv('bond_datas.csv')
        break
    current_page += 1

데이터 크롤링 완료 ! bond_datas.csv에서 확인 가능


In [19]:
# result_df['text'][0]

# 데이터 불러오기 (&전처리)

In [1]:
import pandas as pd
bond_df = pd.read_csv('bond_datas.csv')
bond_df = bond_df[['date', 'text']]

# 전처리한 것
------------------------
시작 Microsoft
Microsoft[^\\]*


Compliance Notice
(당사는)? ?(동|본) ?(조사|자료).*(없습니다|않습니다|됩니다)\.

마지막 '주: 발표.*'  

그림, 표
\<[^\<\>]*\>

그림 [0-9]{1,2}\.
표 [0-9]{1,2}\.

------------------------
Header .*키움증권 리서치센터 투자전략팀
.*팀

ㅎㅎㅎ으로 시작


.*월간채권투자  [0-9]{1,2}월 - 페이지 나누는 기준
Footer 본사·서울시.*

Footer
성명 직급.*
(담당업종 전화 이메일)

필요없는 문장 (자료출처)
자료: [가-힣a-zA-Z, ]* 


이메일
[a-z0-9A-Z]*\@[a-z0-9A-Z]*\.[a-zA-Z]{2,3}\.?[a-zA-Z]{0,2}

---------------------
확인 필요
[가-힣 ]{3,10}[^가-힣]*[0-9]{3,4}\-[0-9]{3,4} [a-zA-Z]

.*Comment

In [106]:
import re
temp = []
for b in bond_df.iloc:
    temp_text = b['text']
    temp_text = temp_text.replace('\n','[!n]')
    temp_text = re.sub('Microsoft[^\[\!n\]]*', '', temp_text) # Header 1
    temp_text = re.sub('.*키움증권 리서치센터 투자전략팀', '', temp_text) # Header 2 (추가확인필요)
    temp_text = re.sub('본사·서울시.*', '', temp_text) # Footer 1
    temp_text = re.sub('성명 직급.*', '', temp_text) # Footer 2
    temp_text = re.sub('Compliance Notice', '', temp_text) # Notice
    temp_text = re.sub('(당사는)? ?(동|본) ?(조사|자료).*(없습니다|않습니다|됩니다|바랍니다)\.?', '', temp_text) # Notice
    temp_text = re.sub('\<[^\<\>]*\>', '', temp_text) # <>사이의 내용
    temp_text = re.sub('그림 [0-9]{1,2}\.', '', temp_text) # 그림
    temp_text = re.sub('표 [0-9]{1,2}\.', '', temp_text) # 표
    temp_text = re.sub('월간채권투자  [0-9]{1,2}월.', '', temp_text) # page 나누는 기준
    temp_text = re.sub('자료: [가-힣a-zA-Z, ]* ?', '', temp_text) # 자료 출처
    temp_text = re.sub('[a-zA-Z0-9\.]*\@[a-zA-Z0-9\.]*', '', temp_text) # 이메일
    temp_text = temp_text.replace('[!n]', ' ')
    temp.append(temp_text)

bond_df['doc'] = temp

In [107]:
bond_df

,date,text,doc
0,20.07.15,Microsoft Word - 2020071416393691.docx\n\n\n \...,1. 회사채 5년물 금리매력 부각 코로나19 확산에 ...
1,20.07.15,2013년 0월 0일\n\n\n \n\n \n\n \n\n \n\n \n\n \n\...,2013년 0월 0일 반쯤...
2,20.07.14,(Microsoft Word - 20200714_Market Inside_\277\...,(nside_\277\354\267\256\265\356\261\336\270\33...
3,20.07.13,\n\n \n\n \n\n \n\n동 자료에 게시된 내용들은 본인의 의견을 정...,  2020년 7월 13일 ㅣ채권전략 ...
4,20.07.10,Microsoft Word - 2020070919412559.docx\n\n\n \...,Credit Investment Maps 2020...
...,...,...,...
3747,08.04.21,Microsoft Word - 920_0421_Fixed Income Weekly....,ncome Weekly.doc 경기 전망은 비교적 분명… ...
3748,08.04.14,Microsoft Word - Fixedincome0414.doc\n\n\n \n\...,ncome0414.doc 4월 금통위는 정부-한은간 pol...
3749,08.04.07,Microsoft Word - 0407.doc\n\n\nFixed Income We...,Fixed Income Weekly 2008. 4. 7. #918 F...
3750,08.04.07,Microsoft Word - 0407.doc\n\n\nFixed Income We...,Fixed Income Weekly 2008. 4. 7. #918 F...


In [112]:
bond_df[['date','doc']].to_json('bond_final.json')

In [70]:
bond_df['doc']

0       \]A등급 3년 이하 발행(좌) A등급 3년 초과 발행(좌) (3년초과/3년이하)배...
1       2013년 0월 0일                                 반쯤...
2                                                   (\]_ 
3                         2020년 7월 13일        ㅣ채권전략 ...
4                                                      \]
                              ...                        
3747    \]도 고객의 증권투자 결과에 대한 법적 책임소재의 증빙자료로 사용될 수 없습니다....
3748                                           \](%p) (%)
3749                                           \](천명) (%)
3750                                           \](천명) (%)
3751    ㅎㅎㅎ   2008_04    월간채권투자  채 권 시 장 전 망  금 융 시 장 ...
Name: doc, Length: 3752, dtype: object

In [ ]:
bond_df

In [74]:
bond_df['text'][2]

'(Microsoft Word - 20200714_Market Inside_\\277\\354\\267\\256\\265\\356\\261\\336\\270\\336\\270\\256\\306\\256_\\(\\277\\317\\))\n\n\n \n\n  1 \n\n \n\n \n\n \n\n  \n\nPoint 1. 주간 신용 스프레드 동향과 전망 \n\nPoint 2. 발행 시장 현황 \n\nPoint 3. 유통 시장 동향 \n\nPoint 4. 신용등급 변동과 아웃룩 현황 \n\n \n\n  \n\n우량등급 장기물  \n투자 메리트 부각 \n\nVol. 24  2020. 7. 14 \n\n\n\n \n\n이베스트투자증권 리서치센터 2 \n\nCredit 이태훈 \n\n02 3779 8479 \n\nthlee@ebestsec.co.kr \n\n \n\nMarket Inside  Market Inside  Market Inside  Market Inside  Credit _ Vol. 24Credit _ Vol. 24Credit _ Vol. 24Credit _ Vol. 24    \n    \n\n  \n\nMarketMarketMarketMarket    InsideInsideInsideInside    CreditCreditCreditCredit        \n\n우량등급 장기물  \n투자 메리트 부각  \n\n \n\nPoint 1. 주간 신용 스프레드 동향과 전망 3 \n\nPoint 2. 발행 시장 현황 7 \n\nPoint 3. 유통 시장 동향 10 \n\nPoint 4. 신용등급 변동과 아웃룩 현황 12 \n\n \n\nContents \n \n\n\n\n \n\n이베스트투자증권 리서치센터 3 \n\nCredit 이태훈 \n\n02 3779 8479 \n\nthlee@ebestsec.co.kr \n\nMarket Inside  Market Inside  Market Inside  Market Inside  Credit _ Vol. 24Credit

In [10]:
bond_df.iloc[3].text

"Compliance Notice \n\n \n\n \n\n\uf06c당사는 동 자료를 기관투자자 또는 제3자에게 사전 제공한 사실이 없습니다. \n\n\uf06c동 자료에 게시된 내용들은 본인의 의견을 정확하게 반영하고 있으며, 외부의 부당한 압력이나 간섭없이 작성되었음을 확인합니다. \n \n\n고지사항  \n\n\uf06c본 조사분석자료는 당사의 리서치센터가 신뢰할 수 있는 자료 및 정보로부터 얻은 것이나, 당사가 그 정확성이나 완전성을 보장할 수 없고, 통지 없이 의견이 변경될 수 \n\n있습니다. \n\uf06c본 조사분석자료는 유가증권 투자를 위한 정보제공을 목적으로 당사 고객에게 배포되는 참고자료로서, 유가증권의 종류, 종목, 매매의 구분과 방법 등에 관한 의사결정은 \n\n전적으로 투자자 자신의 판단과 책임하에 이루어져야 하며, 당사는 본 자료의 내용에 의거하여 행해진 일제의 투자행위 결과에 대하여 어떠한 책임도 지지 않으며 법적 \n\n분쟁에서 증거로 사용 될 수 없습니다. \n\n\uf06c본 조사 분석자료를 무단으로 인용, 복제, 전시, 배포, 전송, 편집, 번역, 출판하는 등의 방법으로 저작권을 침해하는 경우에는 관련법에 의하여 민∙형사상 책임을 지게 \n\n됩니다. \n\n \n\n2020년 7월 13일 \n\n \n키움증권 리서치센터 투자전략팀 \n\nㅣ채권전략 안예하 02) 3787-5157/ yhahn@kiwoom.com \n\nㅣRA 김준수 02)3787-3751 / kjs9460@kiwoom.com \n\n, \n\n금통위 Preview: 채권시장 박스권 유지시킬 금통위 \n \n\nSUMMARY \n\n•7 월 16 일 개최되는 한국은행 금통위에서 기준금리를 0.50%로 만장일치 동결 전망. 수출 지표 등이 \n\n개선되는 가운데, 여전히 코로나 확산 관련 불확실성은 남아있어 현재의 완화적 기조를 유지할 것으로 판단.  \n\n•추가적인 국고채 단순 매입 기대는 여전히 남아 있으나 최근의 단순매입에서 볼 수 있듯 강한 강세 재

In [17]:
bond_df.head(15)

,date,text
0,20.07.15,Microsoft Word - 2020071416393691.docx\n\n\n \...
1,20.07.15,2013년 0월 0일\n\n\n \n\n \n\n \n\n \n\n \n\n \n\...
2,20.07.14,(Microsoft Word - 20200714_Market Inside_\277\...
3,20.07.13,Compliance Notice \n\n \n\n \n\n당사는 동 자료를 기관투...
4,20.07.10,Microsoft Word - 2020070919412559.docx\n\n\n \...
5,20.07.10,2013년 0월 0일\n\n\n \n\n \n\n \n\n \n\n \n\n \n\...
6,20.07.07,(Microsoft Word - 20200707_Market Inside_\273\...
7,20.07.06,Compliance Notice \n\n \n\n \n\n당사는 동 자료를 기관투...
8,20.07.03,Microsoft Word - 2020070220032989.docx\n\n\n \...
9,20.07.03,2013년 0월 0일\n\n\n \n\n \n\n \n\n \n\n \n\n \n\...


In [20]:
bond_df.tail(15)

,date,text
3737,08.05.26,Microsoft Word - 0526_Daewoo Bond Brief.doc\n\...
3738,08.05.26,Microsoft Word - 924_0526_Fixed Income Weekly....
3739,08.05.23,0904fc52801ae375\n\n\n \n\n1 \n\nDaewoo Bond B...
3740,08.05.22,Microsoft Word - Daewoo Bond Brief_0522.doc\n\...
3741,08.05.21,Microsoft Word - Daewoo Bond Brief_0521.doc\n\...
3742,08.05.20,Microsoft Word - 0520_Daewoo Bond Brief.doc\n\...
3743,08.05.19,Microsoft Word - new Bond Brie_080519.doc\n\n\...
3744,08.05.13,Microsoft Word - FixedIncome_0513_.doc\n\n\n \...
3745,08.05.04,ㅎㅎㅎ\n\n\n2008_05 \n\n월간채권투자\n\n채 권 시 장 전 망\n\...
3746,08.04.28,Microsoft Word - FixedIncome_0428.doc\n\n\n \n...


In [11]:
bond_df.iloc[4].text

시작 Microsoft [0-9 \-\_ \\a-zA-Z\.]*
마지막 'Compliance Notice .*'
마지막 주: 발표.*

'Microsoft Word - 2020070919412559.docx\n\n\n \n\n \n\n \n\n \n\n \n\nCredit Investment Maps  2020.7.102020.7.102020.7.102020.7.10\n\n유의미한 우량회사채 5년물 스프레드 축소 \n\n이경록kyoungrok.lee@miraeasset.com   \n\n회사채 수요예측 및 신용등급 � AA-등급의 지에스이피에스는 3개의 트랜치 모두 언더발행 성공. 부정적 등급전망을 \n\n보유하고 있는 AA-등급의 SK인천석유화학은 등급하락 우려에도 불구하고 3개의 \n\n트랜치 모두 증액발행. KB금융은 금리밴드 상단을 이전 발행 대비 소폭 낮췄음에도 \n\n불구하고 3,000억원 모집에 7,930억원이 들어옴 \n\n� AA0등급의 현대제철은 3년물과 5년물에서 각각 2.8배, 4.4배의 유효경쟁률을 \n\n기록하여 증액발행에 하였으며 10년물도 400억원 발행. HDC현대산업개발은 \n\n3,000억원 모집에 110억원만 들어와 미매각 발생. A+등급의 티에스케이코퍼레이션은 \n\n증액발행을 했음에도 불구하고 결정금리가 등급민평 대비 +9bp로 비교적 낮게 정해짐 \n\n \n\n신용스프레드 및 투자전략 � 금주에는 우량회사채 5년물 스프레드 축소가 눈에 띄었는데 확대추세를 보였던 \n\n우량회사채 3-5y 금리차이가 정점을 찍는 느낌  \n\n� 코로나19의 발병이 여전하지만 크레딧 시장의 불안심리는 다소 완화되고 있는 것으로 \n\n판단. 우량등급의 발행은 대체로 성공적이며 A+등급도 기관수요가 늘어나며 \n\n가산금리가 하락. 다만 A0급 이하로는 아직 가산금리가 높고 리테일 수요에 의존하는 \n\n형국이 지속되고 있으며 A급 캐피탈사의 발행도 여의치 않은 상황이 지속  \n\n� AA급 회사채의 가산금리가 안정을 찾아가면서 좀더 높은 금리를 찾아 아래 등급으로 \n\n내려갈 것인지 우량등급의 만기구간을 늘릴 것인지 고민되는 구간에 진입중. 실제로 \n\n3년 초과

In [19]:
bond_df.iloc[6].text

"(Microsoft Word - 20200707_Market Inside_\\273\\363\\271\\335\\261\\342MP\\260\\341\\273\\352_\\(\\277\\317\\))\n\n\n \n\n  1 \n\n \n\n \n\n \n\n  \n\nPoint 1. 크레딧 이슈 \n\nPoint 2. 주간 신용 스프레드 동향과 전망 \n\nPoint 3. 발행 시장 현황 \n\nPoint 4. 유통 시장 동향 \n\nPoint 5. 신용등급 변동과 아웃룩 현황 \n\n \n\n  \n\n등급상승 기대보다  \n등급별, 업종별 \n투자 유효   \n\nVol. 23  2020. 7. 7 \n\n\n\n \n\n이베스트투자증권 리서치센터 2 \n\nCredit 이태훈 \n\n02 3779 8479 \n\nthlee@ebestsec.co.kr \n\n \n\nMarket Inside  Market Inside  Market Inside  Market Inside  Credit _ Vol. 23Credit _ Vol. 23Credit _ Vol. 23Credit _ Vol. 23    \n    \n\n  \n\nMarketMarketMarketMarket    InsideInsideInsideInside    CreditCreditCreditCredit        \n\n등급상승 기대보다  \n등급별, 업종별 투자 유효 \n\n \n\nPoint 1. 크레딧 이슈 3 \n\nPoint 2. 주간 신용 스프레드 동향과 전망 6 \n\nPoint 3. 발행 시장 현황 10 \n\nPoint 4. 유통 시장 동향 13 \n\nPoint 5. 신용등급 변동과 아웃룩 현황 15 \n\n \n\nContents \n \n\n\n\n \n\n이베스트투자증권 리서치센터 3 \n\nCredit 이태훈 \n\n02 3779 8479 \n\nthlee@ebestsec.co.kr \n\nMarket Inside  Market Inside  Market Inside

In [18]:
bond_df.iloc[7].text

Header .*키움증권 리서치센터 투자전략팀


"Compliance Notice \n\n \n\n \n\n\uf06c당사는 동 자료를 기관투자자 또는 제3자에게 사전 제공한 사실이 없습니다. \n\n\uf06c동 자료에 게시된 내용들은 본인의 의견을 정확하게 반영하고 있으며, 외부의 부당한 압력이나 간섭없이 작성되었음을 확인합니다. \n \n\n고지사항  \n\n\uf06c본 조사분석자료는 당사의 리서치센터가 신뢰할 수 있는 자료 및 정보로부터 얻은 것이나, 당사가 그 정확성이나 완전성을 보장할 수 없고, 통지 없이 의견이 변경될 수 \n\n있습니다. \n\uf06c본 조사분석자료는 유가증권 투자를 위한 정보제공을 목적으로 당사 고객에게 배포되는 참고자료로서, 유가증권의 종류, 종목, 매매의 구분과 방법 등에 관한 의사결정은 \n\n전적으로 투자자 자신의 판단과 책임하에 이루어져야 하며, 당사는 본 자료의 내용에 의거하여 행해진 일제의 투자행위 결과에 대하여 어떠한 책임도 지지 않으며 법적 \n\n분쟁에서 증거로 사용 될 수 없습니다. \n\n\uf06c본 조사 분석자료를 무단으로 인용, 복제, 전시, 배포, 전송, 편집, 번역, 출판하는 등의 방법으로 저작권을 침해하는 경우에는 관련법에 의하여 민∙형사상 책임을 지게 \n\n됩니다. \n\n \n\n2020년 7월 6일 \n\n \n키움증권 리서치센터 투자전략팀 \n\nㅣ채권전략 안예하 02) 3787-5157/ yhahn@kiwoom.com \n\nㅣRA 김준수 02)3787-3751 / kjs9460@kiwoom.com \n\n, \n\n한국은행 국고채 단순매입 효과 제한적이지만 \n \n\nSUMMARY \n\n•7 월 2 일, 한국은행은 국고채 단순매입을 1.5 조원 시행. 10 년물 비중이 높아졌다는 점에서 장기 금리 \n\n상승을 안정시키기도 했으나, 한국은행의 자산 규모를 감안한 국채 비중이 큰 변화가 없다는 점에서 \n\n한국은행의 정책 효과에 대한 의구심이 높음 \n\n•다만 외화 유동성 공급 안정을 위해 한국은행과 기재부가 ‘환매조건부 

In [21]:
bond_df.iloc[3745].text

"ㅎㅎㅎ\n\n\n2008_05  \n\n월간채권투자\n\n채 권 시 장 전 망\n\n금 융 시 장 차 트 북\n\n機會의 再構成\n\n\n\n월간채권투자  5월 \n\n \n\n2 \n\n機會의 再構成 \n\n시장이 5월 인하를 기대하며 기회를 만들어 왔으나, 의외의 복병 아닌 복병이 생기고 있다. 그 중 하\n\n나인 미국의 경기바닥 통과론은 사실 별달리 위협적이지 않다. 적어도 상당기간, 가령 한분기 이상은 \n\n말이다. 문제는, 참으로 질긴 인플레 부담이다.  \n\n현실적으로 4월 물가 상승률이 상당히 자극적인 부담요인일 수 있다. 이것을 어떻게 극복해 내는가가 \n\n시장과 일부 금통위원들의 숙제일 것이다. 수요견인 인플레 압력에 대해서는 비교적 낙관한다 해도, \n\n비용충격 측면은 사실 누구도 장담키 어려운 측면이 있다.  \n\n우리는 한 가지 가능성과 한 가지 논리, 그리고 한 가지 대안을 갖고 있다. 우선, 이번 FOMC 이후 \n\n달러가치와 원자재 가격에 일부나마 변화의 조짐이 가시화될 가능성이 없지 않다는 희망섞인 전망을 \n\n유지하고 있다.  \n\n나아가 통화정책은, 누구도 알 수 없는 유가 등락에 일희일비 해서는 안 되며 예측 가능한 인플레 전\n\n망에 따라 정책을 펼쳐야 한다고 생각한다. 금리를 내릴 생각이 아예 없다면 모를까 있는 것이라면, \n\n다달이 유가 변동에 너무 예민해서는 아무런 선제적 조치도 취할 수가 없는 것이다.  \n\n허나 만약에, 이번 금통위 전후 유가가 또 급등해서 인하를 망설이게 된다 하더라도, 6월 금통위에서\n\n의 기회는 여전히, 어쩌면 더 커질 수도 있다는 대안을 품고 있다. \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n달러가치 반등시 원유 선물시장에서의 투기적 매수세 위축될 가능성 : 5월 금통위에 영향 줄 변수 \n\n   \n\n자료: Bloomberg \n \n\n \n\n서철수 02-768-3501 \n\ncyberscs@bestez.com \n\n윤여삼 

In [14]:
bond_df.iloc[3751].text
ㅎㅎㅎ으로 시작


.*월간채권투자  [0-9]{1,2}월 - 페이지 나누는 기준
# Footer 본사·서울시.*

'ㅎㅎㅎ\n\n\n2008_04  \n\n월간채권투자\n\n채 권 시 장 전 망\n\n금 융 시 장 차 트 북\n\n對應과 限界\n\n\n\n월간채권투자  4월 \n\n \n\n2 \n\n對應과 限界 \n\n미국에서는 여전히, 연준은 물론 정부와 의회 역시 신용경색 해소를 위해 다방면의 노력을 하고 있다. \n\n하지만 그것은, 미 금융시스템의 붕괴는 모면케 해 줄 수 있을지언정, 순환적 경기침체까지 피해가게 \n\n만들 수는 없을 것 같다. 금융시장의 유동성이 보강되는 가운데에서도 실물경기는 침체로 가는 상황\n\n이라면, 이는 국내 금리에도 하락 여건일 수 있다고 본다.  \n\n이미 정책이 진행중인 미국과 달리 국내에서는 향후 정책의 방향성을 놓고 당국간 입장차가 확연히 \n\n드러나고 있는 상황이다. 4월 금통위에서도 그다지 금리 우호적 발언이 나오지 않을 리스크를 경계하\n\n고 있는데, 하지만 시장 충격이 그다지 크지는 않을 것으로 보고 있다. 종국적으로 관건은, 객관적 환\n\n경이 누구에게 유리하게 전개되느냐에 있을 것이기 때문이다. 물론 우리는 여전히, 시간은 점차 한은 \n\n편이 아닐 가능성이 우세하다고 보고 있다. \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n이머징 신용위험 약간 둔화되는 가운데 한국 외화 조달 비용 다소 낮아져 \n\n   \n\n자료: Bloomberg \n\n \n\n \n\n서철수 02-768-3501 \n\ncyberscs@bestez.com \n\n윤여삼 02-768-4022 \n\nnersam@bestez.com \n\n윤일광 02-768-3058 \n\nYuniil1@bestez.com \n\n \n\nContent \n\n對應과 限界 ..........................2 \n\n1. 동향 \n\n2. 채권투자환경 \n\n3. 요약과 전망 \n\n금융시장 차트북...................16 \n\n국내금리 동향 \n\n국내금리 